# Goals
**[Script]** Since all companies use public data, the goal is to find gold standard studies quickly

**[Overall]** We want to work with the most reliable, sufficiently powered studies

**[NCBI]: GEO-SRA-Entrez-RefSeq**
- **Note:** For GEO databases, only metadata is stored
    - To retrieve raw data files, a second step is required such as [GEOParse](https://geoparse.readthedocs.io/en/latest/Analyse_hsa-miR-124a-3p_transfection_time-course.html) // [ffq](https://github.com/pachterlab/ffq) // [jq](https://jqlang.org/tutorial/) // [pysradb](https://saket-choudhary.me/pysradb/index.html)

**[EBI]: [ExpressionAtlas](https://www.ebi.ac.uk/gxa/experiments)-[ENA](https://ena-docs.readthedocs.io/en/latest/retrieval/programmatic-access/advanced-search.html)-[Biomart](https://bioconductor.org/packages/release/bioc/vignettes/biomaRt/inst/doc/accessing_ensembl.html)-Ensembl**

**[Retinitis pigmentosa]**
- **[Hypothesis]** HPA links RP to RHO (potential drug target, GPCR, eye-specific)
- **[More]:** [Open Targets](https://platform.opentargets.org/disease/MONDO_0019200/associations?table=associations) // [Unmet Needs](https://pmc.ncbi.nlm.nih.gov/articles/PMC9232096/)

# Packages

In [1]:
#########################
### Standard Library ####
#########################
import os
import re
import sys
import json
import math
import warnings
import subprocess
from glob import glob
from datetime import datetime

#####################
### Data Cleaning ###
#####################
import numpy as np
import pandas as pd
import janitor as jn
import VinlandPy as vp

###################
### Public Data ###
###################
import GEOparse
from Bio import Entrez
from pysradb.sraweb import SRAweb
from pysradb.search import EnaSearch

####################
### Session Info ###
####################
import session_info

/mnt/disks/resources/software/miniconda3/envs/Py-3.12/lib/python3.12/site-packages/pysradb/utils.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/mnt/disks/resources/software/miniconda3/envs/Py-3.12/lib/python3.12/site-packages/requests_ftp/ftp.py:9: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


## Options

In [2]:
warnings.simplefilter(action="once", category=Warning)

pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 200

## Functions

# Parameters

## Inputs
- For more GEO DataSet Types and other query options, see [Query Options](https://www.ncbi.nlm.nih.gov/gds/advanced/)

In [3]:
query_term = "Retinitis pigmentosa"  # (smok* OR diet)
ena_query_term = "Retinitis"  # No hits for "Retinitis pigmentosa"

species = "Homo sapiens"  # (mammals[Organism] NOT human[Organism])
n_samples = "10:1000"

geo_data_types = "expression profiling by high throughput sequencing"
sra_data_types = ["RNA-Seq"]

geo_query = (
    f'{query_term}[All Fields] AND '
    f'{species}[Organism] AND '
    f'{n_samples}[Number of Samples] AND '
    f'"{geo_data_types}"[DataSet Type]'
)
geo_query

'Retinitis pigmentosa[All Fields] AND Homo sapiens[Organism] AND 10:1000[Number of Samples] AND "expression profiling by high throughput sequencing"[DataSet Type]'

## Outputs

In [4]:
input_path = vp.create_dir("./inputs")
search_path = vp.create_dir(os.path.join(input_path, "initial_search"))

../inputs already exists
../outputs already exists
../results already exists
../inputs/initial_search already exists


# Search GEO
- [E-Utils API Access](https://www.ncbi.nlm.nih.gov/geo/info/geo_paccess.html) // [Entrez ESearch](https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&term=Retinitis+pigmentosa[All%20Fields]+AND+Homo%20sapiens[Organism]+AND+10:50[Number%20of%20Samples]&retmax=100000&usehistory=y): query to UIDs // [Entrez ESummary](https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&id=200253526): UIDs to summary record

## List all NCBI databases
- These are the dbs available in NCBI's Entrez

In [5]:
einfo = Entrez.einfo()
ncbi_dbs = Entrez.read(einfo)
einfo.close()
print(sorted(list(ncbi_dbs["DbList"])))

/mnt/disks/resources/software/miniconda3/envs/Py-3.12/lib/python3.12/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


['annotinfo', 'assembly', 'biocollections', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'dbvar', 'gap', 'gapplus', 'gds', 'gene', 'genome', 'geoprofiles', 'grasp', 'gtr', 'ipg', 'medgen', 'mesh', 'nlmcatalog', 'nuccore', 'nucleotide', 'omim', 'orgtrack', 'pcassay', 'pccompound', 'pcsubstance', 'pmc', 'protein', 'proteinclusters', 'protfam', 'pubmed', 'seqannot', 'snp', 'sra', 'structure', 'taxonomy']


## Query GEO

In [6]:
esearch = Entrez.esearch(db="gds", term=geo_query, retmax=100000)  # gds = GEO DataSets
esearch_results = Entrez.read(esearch)
esearch.close()

geo_ids = esearch_results["IdList"]
len(geo_ids)

14

In [7]:
esummary = Entrez.esummary(db="gds", id=",".join(geo_ids))
esummary_results = Entrez.read(esummary)
esummary.close()

df_geo = pd.DataFrame(esummary_results)
vp.print_dims(df_geo, show_rows=(-1, None))

(14, 29)


,Item,Id,Accession,GDS,title,summary,GPL,GSE,taxon,entryType,gdsType,ptechType,valType,SSInfo,subsetInfo,PDAT,suppFile,Samples,Relations,ExtRelations,n_samples,SeriesTitle,PlatformTitle,PlatformTaxa,SamplesTaxa,PubMedIds,Projects,FTPLink,GEO2R
13,[],200062936,GSE62936,,RNA-Seq Analysis in hES/ iPS cell-derived neuronal samples,We characterized the gene expression by Hierarchical Clustering and one-matrix clustering in hES...,16791,62936,Homo sapiens,GSE,Expression profiling by high throughput sequencing,,,,,2014/12/30,CSV,"[{'Accession': 'GSM1536667', 'Title': 'LW04'}, {'Accession': 'GSM1536670', 'Title': 'LW07'}, {'A...",[],"[{'RelationType': 'SRA', 'TargetObject': 'SRP049475', 'TargetFTPLink': 'ftp://ftp-trace.ncbi.nlm...",13,,,,,"[IntegerElement(25555215, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE62nnn/GSE62936/,yes


## Clean GEO

In [8]:
# Aggregate sample info and PMIDs, which are nested
sample_ids = {}
sample_names = {}
pmids = {}
for r in esummary_results:
    ncbi_id = r.get("Id")
    samples = r.get("Samples")
    df_samples = pd.DataFrame(samples)
    
    sample_id = "|".join(df_samples["Accession"].tolist())
    sample_ids[ncbi_id] = sample_id

    sample_name = "|".join(df_samples["Title"].tolist())
    sample_names[ncbi_id] = sample_name

    pmid = list(r.get("PubMedIds", "No PMID"))
    pmid = "|".join([str(int(i)) for i in pmid])
    pmids[ncbi_id] = pmid

In [9]:
# Remove excess columns and add aggregated sample info and PMIDs
geo_cols_to_rm = [
    "Item","GDS","ptechType","valType","SSInfo","subsetInfo","Relations","ExtRelations",
    "SeriesTitle","PlatformTitle","PlatformTaxa","SamplesTaxa","Projects","Samples","PubMedIds"
]
geo_cols_to_rm = [c for c in df_geo.columns if c in geo_cols_to_rm]
df_geo_clean = df_geo.drop(columns=geo_cols_to_rm)

df_geo_clean = df_geo_clean.clean_names(remove_special=True, case_type="lower")
df_geo_clean["sample_ids"] = df_geo_clean["id"].map(sample_ids)
df_geo_clean["sample_names"] = df_geo_clean["id"].map(sample_names)
df_geo_clean["link_to_geo"] = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + df_geo_clean["accession"]
df_geo_clean["link_to_geo"] = df_geo_clean["link_to_geo"].apply(lambda x: f'=HYPERLINK("{x}","Link")')
df_geo_clean["pmids"] = df_geo_clean["id"].map(pmids)

df_geo_clean = df_geo_clean.move(source=["link_to_geo"], target="accession", position="after", axis=1)

# Write out
geo_file_name = f'{datetime.now().strftime("%Y-%m-%d")}-GEO_search'
geo_file = os.path.join(search_path, f"{geo_file_name}.csv")

vp.write_file(df_geo_clean, geo_file)
vp.print_dims(df_geo_clean, show_rows=(-1, None))

Wrote ../inputs/initial_search/2025-06-20-GEO_search.csv
(14, 18)


,id,accession,link_to_geo,title,summary,gpl,gse,taxon,entrytype,gdstype,pdat,suppfile,n_samples,ftplink,geo2r,sample_ids,sample_names,pmids
13,200062936,GSE62936,"=HYPERLINK(""https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE62936"",""Link"")",RNA-Seq Analysis in hES/ iPS cell-derived neuronal samples,We characterized the gene expression by Hierarchical Clustering and one-matrix clustering in hES...,16791,62936,Homo sapiens,GSE,Expression profiling by high throughput sequencing,2014/12/30,CSV,13,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE62nnn/GSE62936/,yes,GSM1536667|GSM1536670|GSM1536673|GSM1536676|GSM1536675|GSM1536672|GSM1536664|GSM1536669|GSM15366...,LW04|LW07|LW11|LW14|LW13|LW10|LW01|LW06|LW03|LW05|LW02|LW09|LW12,25555215


# Alternative data sources
- [recount3 (not used)](https://rna.recount.bio/): 9,000 human and 10,000 mouse studies uniformly processed with GTEx, TCGA, and SRA data

## Search SRA
- **Note:** Some samples are in SRA but not in GEO
- **Note:** Search returns one sample per row (not one study like GEO)

In [10]:
db = SRAweb()
df_sra = db.search_sra(search_str=f'"{query_term}"', assay=True,detailed=True,
                       sample_attribute=True, expand_sample_attributes=True, output_read_lengths=True)

# Filter to species and data types of interest 
df_sra = df_sra[df_sra["organism_name"]==f"{species}"].reset_index(drop=True)
df_sra = df_sra[df_sra.library_strategy.isin(sra_data_types)].reset_index(drop=True)

# Filter to studies with control samples
sra_control_studies = df_sra[df_sra.disease.isin(["Control","normal"])].study_accession.unique().tolist()
df_sra = df_sra[df_sra.study_accession.isin(sra_control_studies)]

# Filter to studies with at least n_samples
sra_powered_studies = df_sra.study_accession.value_counts()
sra_powered_studies = sra_powered_studies[sra_powered_studies > int(n_samples.split(":")[0])].index.tolist()
df_sra = df_sra[df_sra.study_accession.isin(sra_powered_studies)]

# Remove excess columns and clean
sra_cols_to_keep = [
    "study_accession","study_title","run_accession","sample_accession","sample name","biosample",
    "bioproject","phenotype","experiment_title","experiment_accession","instrument_model","patient id",
    "organism_taxid","organism_name","library_name","library_strategy","library_source",
    "library_selection","library_layout","run_total_spots","run_total_bases","source_name","tissue",
    "cell type","genotype","treatment","geo_loc_name","collection_date","cell line","isolate","age",
    "sex","replicate","cell_line","disease","sample_type","common name","disease state","time",
    "ena_fastq_http","ena_fastq_http_1","ena_fastq_http_2"
]
sra_cols_to_keep = [c for c in df_sra.columns if c in sra_cols_to_keep]
df_sra = df_sra[sra_cols_to_keep]

df_sra = df_sra.clean_names(remove_special=True, case_type="lower")
df_sra = df_sra.dropna(axis=1, how="all").replace({pd.NA: None})  # Remove columns with all NAs

# Write out
sra_file_name = f'{datetime.now().strftime("%Y-%m-%d")}-SRA_search'
sra_file = os.path.join(search_path, f"{sra_file_name}.csv")

vp.write_file(df_sra, sra_file)
vp.print_dims(df_sra, show_rows=(None,1))

Wrote ../inputs/initial_search/2025-06-20-SRA_search.csv
(163, 36)


,run_accession,study_accession,study_title,experiment_accession,experiment_title,organism_taxid,organism_name,library_name,library_strategy,library_source,library_selection,library_layout,sample_accession,biosample,bioproject,instrument_model,run_total_spots,run_total_bases,tissue,cell_type,geo_loc_name,collection_date,isolate,age,sex,sex,replicate,cell_line,disease,sample_type,replicate,common_name,sample_name,patient_id,ena_fastq_http_1,ena_fastq_http_2
0,ERR2162231,ERP104602,"RNAseq analysis of fibroblasts, iPSC, iPSC derived RPE and retinal cells from retinitis pigmento...",ERX2218764,Illumina HiSeq 2500 paired end sequencing,9606,Homo sapiens,unspecified,RNA-Seq,TRANSCRIPTOMIC,DNase,PAIRED,ERS1960817,SAMEA104335839,PRJEB22885,Illumina HiSeq 2500,31153763,6253531827,None,fibroblast,None,None,None,None,None,None,None,None,PRPF31,None,None,human,PRPF31001,F116,http://ftp.sra.ebi.ac.uk/vol1/fastq/ERR216/001/ERR2162231/ERR2162231_1.fastq.gz,http://ftp.sra.ebi.ac.uk/vol1/fastq/ERR216/001/ERR2162231/ERR2162231_2.fastq.gz


## Search ENA

In [11]:
ena_search = EnaSearch(return_max=1000, query=ena_query_term)
ena_search.search()
df_ena = ena_search.get_df()

# Filter to species and data types of interest 
df_ena = df_ena[df_ena.scientific_name==f"{species}"].reset_index(drop=True)
df_ena = df_ena[df_ena.library_strategy.isin(sra_data_types)].reset_index(drop=True)
df_ena = df_ena.dropna(axis=1, how="all")

# Write out
ena_file_name = f'{datetime.now().strftime("%Y-%m-%d")}-ENA_search'
ena_file = os.path.join(search_path, f"{ena_file_name}.csv")

vp.write_file(df_ena, ena_file)
vp.print_dims(df_ena, show_rows=(None,1))

Wrote ../inputs/initial_search/2025-06-20-ENA_search.csv
(10, 15)


,study_accession,experiment_accession,experiment_title,description,tax_id,scientific_name,library_strategy,library_source,library_selection,sample_accession,sample_title,instrument_model,run_accession,read_count,base_count
0,PRJEB48601,ERX6798797,NextSeq 500 sequencing: Retinitis pigmentosa-linked mutation in DHX38 modulates its splicing act...,NextSeq 500 sequencing: Retinitis pigmentosa-linked mutation in DHX38 modulates its splicing act...,9606,Homo sapiens,RNA-Seq,TRANSCRIPTOMIC,cDNA_randomPriming,SAMEA10774731,DHX38_3,NextSeq 500,ERR7229434,75095552,5597285638


## Search ExpressionAtlas
Since there are not a lot of studies here, we can search this through the web browser - [bulkEA](https://www.ebi.ac.uk/gxa/experiments?species=Homo+sapien) + [scEA](https://www.ebi.ac.uk/gxa/sc/experiments?species=%22homo+sapiens%22)
- Queries can be either in a baseline context, e.g. find genes expressed in the macaque brain, or in a differential context, e.g. find genes that are up or downregulated in response to auxin in Arabidopsis
- All studies are manually curated and annotated with ontology terms, checked for high quality and re-analysed using standardised methods

In [12]:
print("Use web browser")

Use web browser


## Search DISCO
- **Note:** [DISCO](https://disco.bii.a-star.edu.sg/) is one of the best resources for scRNAseq data. Also see [SCimilarity](https://genentech.github.io/scimilarity/index.html).

# Write to Excel
- **Note:** Each tab must be created and formatted first before writing them out together as an Excel file

In [14]:
excel_file_name = f'{datetime.now().strftime("%Y-%m-%d")}_Public_Studies_Initial_Search'
excel_file = os.path.join("./", f"{excel_file_name}.xlsx")

geo_headers = vp.get_headers(df_geo_clean, colors=["F7FFBF"])
wb = vp.create_excel_tab(
        df_geo_clean,
        geo_headers,
        tab_name="GEO",
        out_file=None,
        previous_workbook=None,
        template_file=None,
        freeze_cell="A2",
        row_to_group=None,
        cnd_format_dict={"link_to_geo": "link-0000FF-single"})

sra_headers = vp.get_headers(df_sra, colors=["F7FFBF"])
wb = vp.create_excel_tab(
        df_sra,
        sra_headers,
        tab_name="SRA",
        out_file=None,
        previous_workbook=wb,
        template_file=None,
        freeze_cell="A2",
        row_to_group=None,
        cnd_format_dict=None)

ena_headers = vp.get_headers(df_ena, colors=["F7FFBF"])
wb = vp.create_excel_tab(
        df_ena,
        ena_headers,
        tab_name="ENA",
        out_file=excel_file,
        previous_workbook=wb,
        template_file=None,
        freeze_cell="A2",
        row_to_group=None,
        cnd_format_dict=None)

Wrote *** ../results/2025-06-20_Public_Studies_Initial_Search.xlsx ***


# Session info

In [3]:
session_info.show(os=True, std_lib=False, dependencies=False)